## The analysis of patent
### 主要是分析企业的专利数量、研发投入等，借此分析企业的创新水平

In [ ]:
# 根据国标-国家经济行业分类 https://www.mca.gov.cn/images3/www/file/201711/1509495881341.pdf
# 进行patent初步筛选: 国民经济行业分类代码,国民经济行业分类名称

from collections import Counter
import json
from tqdm import tqdm
import os
from patent_utils import get_output_file_name
from patent_utils import is_chinese

input_directory = '/mntmodels/boyuan/learnspace/AI_gov/raw_data/research_data/patent_data_all/data_json_filtered'
output_directory = '/mntmodels/boyuan/learnspace/AI_gov/raw_data/research_data/patent_data_all/data_json_GB_filtered'
os.makedirs(output_directory, exist_ok=True)

def filter_GB(category_code):
    # 采矿业
    # print(type(category_code))
    if category_code > 600 and category_code < 1300:
        return 1
    # 制造业
    if category_code > 3200 and category_code < 4200:
        return 1
    return 0


def check(patent_data):
    '''
    检查Patent Data 中是否有违规的数据
    '''
    # 检查筛选键值是否存在异常取值
    key='国民经济行业分类代码'
    values = [item[key] for item in patent_data if key in item]

    counter = Counter(values)

    print(counter)
    print(len(counter))

    # 检查int转化是否有效
    entry = patent_data[0]
    print(int(entry['国民经济行业分类代码']))
    print(entry['国民经济行业分类代码'])


for root, dirs, files in os.walk(input_directory):
    for file in tqdm(files):
        if file.endswith('.json'):
            patent_input = os.path.join(root, file)
        else:
            continue
        print('\n\n')
        print('**'*60)
        with open(patent_input,'r') as f:
            patent_data = json.load(f)
        check(patent_data)
        new_patent_data = []
        flag = 1
        for entry in patent_data:
            # 有些数据里存在该键值为空或者不是float是str的情况，需要进行额外筛选
            if entry['国民经济行业分类代码'] is None:
                continue
            
            # 检查是否为中文字符串
            if type(entry ['国民经济行业分类代码']) is str and is_chinese(entry['国民经济行业分类代码']):
                continue
            
            try:
                category_id_float = float(entry['国民经济行业分类代码'])
                # 可以在这里添加转换成功后的处理，例如添加到新列表中
                if filter_GB(int(category_id_float)) == 1:
                    new_patent_data.append(entry)
            except ValueError:
                # 如果转换失败（即不是有效的数字字符串），可以在这里处理或者跳过
                continue
                
        print(f'Total Number for original data: {len(patent_data)}')
        print(f'Total Number after GB filterd:{len(new_patent_data)}')

        # 构造输出文件的完整路径
        output_file_name = get_output_file_name(file)
        output_file_path = os.path.join(output_directory, output_file_name)
        
        # 将提取的数据保存到新的JSON文件中
        with open(output_file_path, 'w', encoding='utf-8') as output_file:
            json.dump(new_patent_data, output_file, ensure_ascii=False, indent=4)

        print(f"Filtered data have been saved to {output_file_path}")
        print('**'*60) 


In [ ]:
# 再进行完GB筛选后，首先进行一波数据分析
# 分析申请人的组成成分,比如企业 \ 个人 \ NoneType等
# 这里最后可以出一个热力图，统计各个patent数据的键值分布情况,同时把其他的键值不对的情况处理了（比如申请人类型是日期的）
# 统计一下原始数据分布情况和筛选后数据分布情况

from collections import Counter
import json


input_directory = '/mntmodels/boyuan/learnspace/AI_gov/raw_data/research_data/patent_data_all/data_json_GB_filtered'

for root, dirs, files in os.walk(input_directory):
    for file in tqdm(files):
        if file.endswith('.json'):
            patent_input = os.path.join(root, file)
        else:
            continue
        print('\n\n')
        print('**'*60)
        with open(patent_input,'r') as f:
            patent_data = json.load(f)
        key = '申请人类型'
        values = [item[key] for item in patent_data if key in item]
        counter = Counter(values)
        print(counter)
        print('**'*60)


        

### 由结果惊喜的发现，之前看到的申请人类型是nonetype的，在筛选国民经济行业分类代码的时候同步也被筛掉了

In [4]:
# 接下来进行正式的处理，采取专利匹配操作和知识图谱构建

# 首先从工商企业数据得到作参考的name_list
import json

reference_input = '/mntmodels/boyuan/learnspace/AI_gov/raw_data/research_data/business_info/output_extracted_new.json'
with open(reference_input,'r') as f:
    reference_data = json.load(f)
name_list = []
for entry in reference_data:
    name_list.append(
        entry['origin_info']['name']
    )
print(name_list[0:10])
print(f'Total Number for namelist:{len(name_list)}')

['武汉市鑫力荣物资再生有限公司', '武汉滨水东熠进出口服务有限公司', '武汉品瀚电子商务商行', '武汉文鸿汽车服务有限公司', '湖北华屹企业管理咨询有限公司', '湖北振意贸易有限公司', '湖北鑫润旺六口茶农业科技有限公司', '武汉柏雍电子商务商行', '湖北民生石油液化气有限公司武汉常码头加气站', '武汉康通运输有限公司']
Total Number for namelist:97810


### 分析：为什么专利数据这么少，其实大概就是少：
### 1. 专利数据是国家专利局的数据，不是光谷的数据，所以其实还要做一个潜在筛选武汉的公司（其实直接和武汉市企业工商信息匹配就可以了）
### 2. 大概可以去遍历所有的专利数据，最后只保留总的，不用每年都保留

In [6]:

# 进行正式的数据分析，根据上面得到的name_list 过所有的GB_filtered json files 筛选出所有‘武汉光谷’的专利申请数据

from collections import Counter
import json
import os
from tqdm import tqdm

input_directory = '/mntmodels/boyuan/learnspace/AI_gov/raw_data/research_data/patent_data_all/data_json_GB_filtered'

patent_enter_data = []
for root, dirs, files in os.walk(input_directory):
    for file in files:
        if file.endswith('.json'):
            patent_input = os.path.join(root, file)
        else:
            continue
        with open(patent_input,'r') as f:
            patent_data = json.load(f)
        cnt = 0
        for patent_entry in tqdm(patent_data):
            if patent_entry['主申请人名称'] in name_list:
                patent_enter_data.append(patent_entry)
                cnt += 1
        print(f'Total Number for patent - enterprise: {cnt}')

100%|██████████| 359309/359309 [07:31<00:00, 796.06it/s]


Total Number for patent - enterprise: 734


100%|██████████| 281426/281426 [05:58<00:00, 784.65it/s]


Total Number for patent - enterprise: 281


100%|██████████| 257725/257725 [05:28<00:00, 784.93it/s]


...

Total Number for patent - enterprise: 247


100%|██████████| 80374/80374 [01:38<00:00, 811.93it/s]


Total Number for patent - enterprise: 19


100%|██████████| 308591/308591 [06:24<00:00, 802.61it/s]

Total Number for patent - enterprise: 556


In [2]:
import json
output_file = '/mntmodels/boyuan/learnspace/AI_gov/raw_data/research_data/patent_data_all/patent_final/patent_final.json'
with open(output_file,'w',encoding='utf-8') as f:
    json.dump(patent_enter_data,f,indent=2,ensure_ascii=False)


NameError: name 'patent_enter_data' is not defined